In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import re
from collections import Counter
import liwc
import math
import json
import os
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import arabic_reviews

In [2]:
data = pd.read_json("EnglishBookReviews.json")

In [4]:
data.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,12889ca3ea33be08b182e1bab60e8b55,13087982,986067c40edc87e152e4ec317c4b56c3,5,I don't even know how to review this. I feel l...,Mon Mar 03 14:49:09 -0800 2014,Tue Sep 19 20:42:44 -0700 2017,Thu Jan 19 00:00:00 -0800 2017,Sun Sep 20 00:00:00 -0700 2015,3,0
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,,,1,0
2,6ba3427be7a706f83d079e8ec902a48d,14061957,d6f80a1b3eaee0b640d5a99e1e7c601b,5,"""Maybe love was superstition, a prayer we said...",Mon Sep 28 22:00:23 -0700 2015,Sun Oct 18 19:10:31 -0700 2015,Mon Sep 28 00:00:00 -0700 2015,Mon Sep 28 00:00:00 -0700 2015,1,0
3,424956bc779a611313ff39813ed4ca14,35181314,ec7d733d477cee5cd377eea238fea747,5,Review to come,Tue Apr 04 10:56:56 -0700 2017,Fri Oct 20 11:11:43 -0700 2017,Fri Oct 20 14:12:21 -0700 2017,Thu Sep 28 23:39:33 -0700 2017,1,0
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [12]:
data_conf = data[:10000]
data_conf["helpfulness"] = data['n_votes'][:10000].map(lambda x: "helpful" if x > 0 else "unhelpful")
classes = data_conf["helpfulness"]

d_h = data_conf[data_conf["helpfulness"] == "helpful"]
len(d_h)

d_u = data_conf[data_conf["helpfulness"] != "helpful"]
len(d_h)

print(len(d_h))
print(len(d_u))

4098
5902


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
tf_idf = arabic_reviews.extract_tfidf(data_conf["review_text"])

[nltk_data] Downloading package stopwords to /Users/Omar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Omar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Sample feature names identified :  ['***arc', '***copy', '***review', '***this', '**arc', '**review', '*_*', '*a', '*arc', '*cough', '*cough*', '*cries*', '*cue', '*deep', '*disclaimer', '*facepalm*', '*fans', '*fingers', '*gasp*', '*goes', '*i', '*note', '*really*', '*review', '*rolls']

Size of TFIDF matrix :  (10000, 15429)


In [14]:
tf_idf[0].shape

(10000, 15429)

In [15]:
classes = data_conf["helpfulness"].map(lambda x: "not_helpful" if x != "helpful" else "helpful")

In [16]:
arabic_reviews.ex_with_best_k(tf_idf[0],classes,100)

k = ----------  1
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  0 401]
 [  0 599]]
Acc:  0.599  Prec:  0.599  Recall:  1.0  F1: 0.749
SVM
Confusion Matrix : 
[[  2 399]
 [  0 599]]
Acc:  0.601  Prec:  0.6  Recall:  1.0  F1: 0.75
Decision Tree
Confusion Matrix : 
[[  6 395]
 [  1 598]]
Acc:  0.604  Prec:  0.602  Recall:  0.998  F1: 0.751
Random Forest
Confusion Matrix : 
[[ 10 391]
 [  1 598]]
Acc:  0.608  Prec:  0.605  Recall:  0.998  F1: 0.753
NN
Confusion Matrix : 
[[  8 393]
 [  0 599]]
Acc:  0.607  Prec:  0.604  Recall:  1.0  F1: 0.753
k = ----------  2
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  0 401]
 [  0 599]]
Acc:  0.599  Prec:  0.599  Recall:  1.0  F1: 0.749
SVM
Confusion Matrix : 
[[  3 398]
 [  1 598]]
Acc:  0.601  Prec:  0.6  Recall:  0.998  F1: 0.75
Decision Tree
Confusion Matrix : 
[[ 11 390]
 [  4 595]]
Acc:  0.606  Pre

SVM
Confusion Matrix : 
[[ 17 384]
 [  7 592]]
Acc:  0.609  Prec:  0.607  Recall:  0.988  F1: 0.752
Decision Tree
Confusion Matrix : 
[[111 290]
 [116 483]]
Acc:  0.594  Prec:  0.625  Recall:  0.806  F1: 0.704
Random Forest
Confusion Matrix : 
[[ 45 356]
 [ 23 576]]
Acc:  0.621  Prec:  0.618  Recall:  0.962  F1: 0.752
NN
Confusion Matrix : 
[[100 301]
 [ 65 534]]
Acc:  0.634  Prec:  0.64  Recall:  0.891  F1: 0.745
k = ----------  15
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  0 401]
 [  0 599]]
Acc:  0.599  Prec:  0.599  Recall:  1.0  F1: 0.749
SVM
Confusion Matrix : 
[[ 18 383]
 [  7 592]]
Acc:  0.61  Prec:  0.607  Recall:  0.988  F1: 0.752
Decision Tree
Confusion Matrix : 
[[110 291]
 [117 482]]
Acc:  0.592  Prec:  0.624  Recall:  0.805  F1: 0.703
Random Forest
Confusion Matrix : 
[[ 39 362]
 [ 21 578]]
Acc:  0.617  Prec:  0.615  Recall:  0.965  F1: 0.751
NN
Confusion Matrix : 
[[114 287]
 [ 71 528]]
Acc:  0

Confusion Matrix : 
[[151 250]
 [177 422]]
Acc:  0.573  Prec:  0.628  Recall:  0.705  F1: 0.664
Random Forest
Confusion Matrix : 
[[ 46 355]
 [ 25 574]]
Acc:  0.62  Prec:  0.618  Recall:  0.958  F1: 0.751
NN
Confusion Matrix : 
[[  0 401]
 [  0 599]]
Acc:  0.599  Prec:  0.599  Recall:  1.0  F1: 0.749
k = ----------  28
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  1 400]
 [  1 598]]
Acc:  0.599  Prec:  0.599  Recall:  0.998  F1: 0.749
SVM
Confusion Matrix : 
[[ 27 374]
 [ 10 589]]
Acc:  0.616  Prec:  0.612  Recall:  0.983  F1: 0.754
Decision Tree
Confusion Matrix : 
[[152 249]
 [181 418]]
Acc:  0.57  Prec:  0.627  Recall:  0.698  F1: 0.66
Random Forest
Confusion Matrix : 
[[ 47 354]
 [ 30 569]]
Acc:  0.616  Prec:  0.616  Recall:  0.95  F1: 0.748
NN
Confusion Matrix : 
[[121 280]
 [ 78 521]]
Acc:  0.642  Prec:  0.65  Recall:  0.87  F1: 0.744
k = ----------  29
Classes found :  ['helpful' 'not_helpful']

Classes c

Random Forest
Confusion Matrix : 
[[ 41 360]
 [ 17 582]]
Acc:  0.623  Prec:  0.618  Recall:  0.972  F1: 0.755
NN
Confusion Matrix : 
[[126 275]
 [100 499]]
Acc:  0.625  Prec:  0.645  Recall:  0.833  F1: 0.727
k = ----------  41
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  3 398]
 [  2 597]]
Acc:  0.6  Prec:  0.6  Recall:  0.997  F1: 0.749
SVM
Confusion Matrix : 
[[ 30 371]
 [ 14 585]]
Acc:  0.615  Prec:  0.612  Recall:  0.977  F1: 0.752
Decision Tree
Confusion Matrix : 
[[157 244]
 [186 413]]
Acc:  0.57  Prec:  0.629  Recall:  0.689  F1: 0.658
Random Forest
Confusion Matrix : 
[[ 43 358]
 [ 16 583]]
Acc:  0.626  Prec:  0.62  Recall:  0.973  F1: 0.757
NN
Confusion Matrix : 
[[  0 401]
 [  0 599]]
Acc:  0.599  Prec:  0.599  Recall:  1.0  F1: 0.749
k = ----------  42
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  3 398]
 [  2 597]]
Acc:  0.6

NN
Confusion Matrix : 
[[182 219]
 [184 415]]
Acc:  0.597  Prec:  0.655  Recall:  0.693  F1: 0.673
k = ----------  54
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  4 397]
 [  2 597]]
Acc:  0.601  Prec:  0.601  Recall:  0.997  F1: 0.75
SVM
Confusion Matrix : 
[[ 38 363]
 [ 16 583]]
Acc:  0.621  Prec:  0.616  Recall:  0.973  F1: 0.755
Decision Tree
Confusion Matrix : 
[[171 230]
 [181 418]]
Acc:  0.589  Prec:  0.645  Recall:  0.698  F1: 0.67
Random Forest
Confusion Matrix : 
[[ 44 357]
 [ 21 578]]
Acc:  0.622  Prec:  0.618  Recall:  0.965  F1: 0.754
NN
Confusion Matrix : 
[[127 274]
 [116 483]]
Acc:  0.61  Prec:  0.638  Recall:  0.806  F1: 0.712
k = ----------  55
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  4 397]
 [  2 597]]
Acc:  0.601  Prec:  0.601  Recall:  0.997  F1: 0.75
SVM
Confusion Matrix : 
[[ 38 363]
 [ 16 583]]
Acc:  0.621  Pr

SVM
Confusion Matrix : 
[[ 36 365]
 [ 15 584]]
Acc:  0.62  Prec:  0.615  Recall:  0.975  F1: 0.755
Decision Tree
Confusion Matrix : 
[[176 225]
 [182 417]]
Acc:  0.593  Prec:  0.65  Recall:  0.696  F1: 0.672
Random Forest
Confusion Matrix : 
[[ 41 360]
 [ 15 584]]
Acc:  0.625  Prec:  0.619  Recall:  0.975  F1: 0.757
NN
Confusion Matrix : 
[[125 276]
 [128 471]]
Acc:  0.596  Prec:  0.631  Recall:  0.786  F1: 0.7
k = ----------  68
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  4 397]
 [  2 597]]
Acc:  0.601  Prec:  0.601  Recall:  0.997  F1: 0.75
SVM
Confusion Matrix : 
[[ 36 365]
 [ 15 584]]
Acc:  0.62  Prec:  0.615  Recall:  0.975  F1: 0.755
Decision Tree
Confusion Matrix : 
[[177 224]
 [184 415]]
Acc:  0.592  Prec:  0.649  Recall:  0.693  F1: 0.67
Random Forest
Confusion Matrix : 
[[ 45 356]
 [ 22 577]]
Acc:  0.622  Prec:  0.618  Recall:  0.963  F1: 0.753
NN
Confusion Matrix : 
[[135 266]
 [123 476]]
Acc:  0.61

Acc:  0.625  Prec:  0.619  Recall:  0.975  F1: 0.757
Decision Tree
Confusion Matrix : 
[[169 232]
 [183 416]]
Acc:  0.585  Prec:  0.642  Recall:  0.694  F1: 0.667
Random Forest
Confusion Matrix : 
[[ 32 369]
 [ 12 587]]
Acc:  0.619  Prec:  0.614  Recall:  0.98  F1: 0.755
NN
Confusion Matrix : 
[[144 257]
 [131 468]]
Acc:  0.612  Prec:  0.646  Recall:  0.781  F1: 0.707
k = ----------  81
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  4 397]
 [  2 597]]
Acc:  0.601  Prec:  0.601  Recall:  0.997  F1: 0.75
SVM
Confusion Matrix : 
[[ 41 360]
 [ 15 584]]
Acc:  0.625  Prec:  0.619  Recall:  0.975  F1: 0.757
Decision Tree
Confusion Matrix : 
[[164 237]
 [177 422]]
Acc:  0.586  Prec:  0.64  Recall:  0.705  F1: 0.671
Random Forest
Confusion Matrix : 
[[ 33 368]
 [ 11 588]]
Acc:  0.621  Prec:  0.615  Recall:  0.982  F1: 0.756
NN
Confusion Matrix : 
[[128 273]
 [120 479]]
Acc:  0.607  Prec:  0.637  Recall:  0.8  F1: 0.709
k 

Random Forest
Confusion Matrix : 
[[ 34 367]
 [  9 590]]
Acc:  0.624  Prec:  0.617  Recall:  0.985  F1: 0.758
NN
Confusion Matrix : 
[[144 257]
 [122 477]]
Acc:  0.621  Prec:  0.65  Recall:  0.796  F1: 0.716
k = ----------  94
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  3 398]
 [  2 597]]
Acc:  0.6  Prec:  0.6  Recall:  0.997  F1: 0.749
SVM
Confusion Matrix : 
[[ 44 357]
 [ 17 582]]
Acc:  0.626  Prec:  0.62  Recall:  0.972  F1: 0.757
Decision Tree
Confusion Matrix : 
[[166 235]
 [189 410]]
Acc:  0.576  Prec:  0.636  Recall:  0.684  F1: 0.659
Random Forest
Confusion Matrix : 
[[ 34 367]
 [ 17 582]]
Acc:  0.616  Prec:  0.613  Recall:  0.972  F1: 0.752
NN
Confusion Matrix : 
[[152 249]
 [142 457]]
Acc:  0.609  Prec:  0.647  Recall:  0.763  F1: 0.7
k = ----------  95
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [0 0 0 ... 1 0 0]
NB
Confusion Matrix : 
[[  3 398]
 [  2 597]]
Acc:  0.6

In [163]:
top_words(tf_idf[1],tf_idf[0],classes,70,"english_books_top_words.csv")

[('sexy', 7.274543071508593),
 ('provided', 3.4732246834284366),
 ('return', 2.640942178386892),
 ('ms.', 2.593428178638642),
 ('review', 2.5183544240999396),
 ('jake', 2.4555657504936965),
 ('author', 2.4127694403411746),
 ('honest', 2.344010199071998),
 ('arc', 2.244466040402421),
 ('brooke', 2.0831827039140642),
 ('romance', 2.0711872978220116),
 ('hea', 1.966336476902383),
 ('meet', 1.7524179717443575),
 ('hot', 1.7107509419006668),
 ('rachel', 1.6754243645196254),
 ('kacey', 1.6448366688969465),
 ('caleb', 1.558468110544505),
 ('tate', 1.5577249809635467),
 ('alpha', 1.534186020493805),
 ('biker', 1.5242344997835748),
 ('maas', 1.507273270109137),
 ('december', 1.4974671073992707),
 ('eve', 1.4553605083056396),
 ('every', 1.4221203761299064),
 ('eventually', 1.4158302202403725),
 ('fresh', 1.410458890238579),
 ('wes', 1.3962859296209835),
 ('come', 1.3599904592928906),
 ('douglas', 1.3494710377140329),
 ('away', 1.341141013745772),
 ('honor', 1.3344549808677328),
 ('amelia', 1.320

# A General Classifier 

In [164]:
data_conf.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,helpfulness
0,12889ca3ea33be08b182e1bab60e8b55,13087982,986067c40edc87e152e4ec317c4b56c3,5,I don't even know how to review this. I feel l...,Mon Mar 03 14:49:09 -0800 2014,Tue Sep 19 20:42:44 -0700 2017,Thu Jan 19 00:00:00 -0800 2017,Sun Sep 20 00:00:00 -0700 2015,3,0,helpful
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,,,1,0,helpful
2,6ba3427be7a706f83d079e8ec902a48d,14061957,d6f80a1b3eaee0b640d5a99e1e7c601b,5,"""Maybe love was superstition, a prayer we said...",Mon Sep 28 22:00:23 -0700 2015,Sun Oct 18 19:10:31 -0700 2015,Mon Sep 28 00:00:00 -0700 2015,Mon Sep 28 00:00:00 -0700 2015,1,0,helpful
3,424956bc779a611313ff39813ed4ca14,35181314,ec7d733d477cee5cd377eea238fea747,5,Review to come,Tue Apr 04 10:56:56 -0700 2017,Fri Oct 20 11:11:43 -0700 2017,Fri Oct 20 14:12:21 -0700 2017,Thu Sep 28 23:39:33 -0700 2017,1,0,helpful
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1,helpful


In [165]:
english_books = data_conf

In [166]:
arabic_books = pd.read_csv("appen_translated - books.csv")

In [167]:
arabic_books.head()


,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,review_en
0,2905523768,False,finalized,3,12/23/2020 7:25:58,somewhat_helpful,0.6598,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,Whatever the intellectual tastes .. necessary ...
1,2905523769,False,finalized,3,12/23/2020 7:25:58,helpful,0.6598,52854,.. .. هنا التقيت بأحد أعجب الرجال اطلاقا،شمس ا...,.. .. here I met one impressed by the men at a...
2,2905523770,False,finalized,3,12/23/2020 7:25:58,unrelated,0.6598,48057,سأعود,I'll come back
3,2905523771,False,finalized,3,12/23/2020 7:25:58,helpful,1.0000,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,I had not read the dreams of the famous Thelat...
4,2905523772,False,finalized,3,12/23/2020 7:54:33,helpful,1.0000,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,Language fascinates me and her pen. I wished t...


In [168]:
arabic_books_conf = arabic_books[(arabic_books["confidence"] == 1.0) & (arabic_books["helpfulness"] == "helpful") | (arabic_books["helpfulness"] != "helpful")]

In [169]:
arabic_books_conf["helpfulness"] = arabic_books_conf["helpfulness"].map(lambda x: "not_helpful" if x != "helpful" else "helpful")

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [170]:
arabic_books_conf.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,helpfulness,confidence,no,review,review_en
0,2905523768,False,finalized,3,12/23/2020 7:25:58,not_helpful,0.6598,64963,مهما كانت ميولك الفكريه .. لازم هتحب حاجه في ا...,Whatever the intellectual tastes .. necessary ...
2,2905523770,False,finalized,3,12/23/2020 7:25:58,not_helpful,0.6598,48057,سأعود,I'll come back
3,2905523771,False,finalized,3,12/23/2020 7:25:58,helpful,1.0000,14611,لم أكن قد قرأتُ لأحلام ثلاثيتها الشهيرة، ولم أ...,I had not read the dreams of the famous Thelat...
4,2905523772,False,finalized,3,12/23/2020 7:54:33,helpful,1.0000,17368,تسحرني لغتها وقلمها. تمنيت أن صفحات الكتاب لا ...,Language fascinates me and her pen. I wished t...
5,2905523773,False,finalized,3,12/23/2020 7:54:33,not_helpful,0.6598,59497,مطولة بشكل مبالغ، لكن تلقي الضوء على جوانب غير...,"Excessively lengthy, but shed light on aspects..."


In [174]:
ar_books_simplified = arabic_books_conf[["review_en","helpfulness"]]

In [175]:
ar_books_simplified.head()

,review_en,helpfulness
0,Whatever the intellectual tastes .. necessary ...,not_helpful
2,I'll come back,not_helpful
3,I had not read the dreams of the famous Thelat...,helpful
4,Language fascinates me and her pen. I wished t...,helpful
5,"Excessively lengthy, but shed light on aspects...",not_helpful


In [176]:
english_books_simplified = english_books[["review_text","helpfulness"]]

In [177]:
english_books_simplified.head()

,review_text,helpfulness
0,I don't even know how to review this. I feel l...,helpful
1,Recommended by Don Katz. Avail for free in Dec...,helpful
2,"""Maybe love was superstition, a prayer we said...",helpful
3,Review to come,helpful
4,"I really enjoyed this book, and there is a lot...",helpful


In [181]:
english_books_simplified.columns = ["review_en","helpfulness"]

In [182]:
english_books_simplified.head()

,review_en,helpfulness
0,I don't even know how to review this. I feel l...,helpful
1,Recommended by Don Katz. Avail for free in Dec...,helpful
2,"""Maybe love was superstition, a prayer we said...",helpful
3,Review to come,helpful
4,"I really enjoyed this book, and there is a lot...",helpful


In [208]:
combined = pd.concat([ar_books_simplified,english_books_simplified])

In [209]:
combined.head()

,review_en,helpfulness
0,Whatever the intellectual tastes .. necessary ...,not_helpful
2,I'll come back,not_helpful
3,I had not read the dreams of the famous Thelat...,helpful
4,Language fascinates me and her pen. I wished t...,helpful
5,"Excessively lengthy, but shed light on aspects...",not_helpful


In [210]:
combined.groupby("helpfulness").describe()

review_en         \
                count unique   
helpfulness                    
helpful          1470   1470   
not_helpful       662    659   
unhelpful         828    827   

                                                                     
                                                           top freq  
helpfulness                                                          
helpful      Oh oh. Uh what carries the novel of the manife...    1  
not_helpful                                          Fantastic    3  
unhelpful                  3.5 stars \n Listened to audiobook.    2

In [211]:
len(combined[combined["helpfulness"]=="unhelpful"])

828

In [212]:
combined["helpfulness"] = combined["helpfulness"].map(lambda x: "not_helpful" if x == "unhelpful" or x == "not_helpful" else "helpful")

In [215]:
import arabic_reviews
tf_idf_combined = arabic_reviews.extract_tfidf(combined["review_en"])

[nltk_data] Downloading package stopwords to /Users/Omar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Omar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Sample feature names identified :  ['****some', '****spoiler', '***thanks', '**excellent', '**final', '**review', '**swoon**', '**the', '**update**', '**you', '*a', '*addictive*', '*an', '*arc', '*are*', '*bitterly', '*but*', '*cough', '*cough*', '*cough-noah', '*crying*', '*edited', '*eye', '*facepalm*', '*fans']

Size of TFIDF matrix :  (2960, 23270)


In [216]:
classes = combined["helpfulness"]

In [217]:
arabic_reviews.ex_with_best_k(tf_idf_combined[0],classes,100)

k = ----------  1
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[138   0]
 [158   0]]
Acc:  0.466  Prec:  0.0  Recall:  0.0  F1: 0.0
SVM
Confusion Matrix : 
[[ 47  91]
 [ 24 134]]
Acc:  0.611  Prec:  0.596  Recall:  0.848  F1: 0.7
Decision Tree
Confusion Matrix : 
[[ 59  79]
 [ 33 125]]
Acc:  0.622  Prec:  0.613  Recall:  0.791  F1: 0.691


/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest
Confusion Matrix : 
[[ 74  64]
 [ 38 120]]
Acc:  0.655  Prec:  0.652  Recall:  0.759  F1: 0.702
NN
Confusion Matrix : 
[[ 74  64]
 [ 37 121]]
Acc:  0.659  Prec:  0.654  Recall:  0.766  F1: 0.706
k = ----------  2
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 85  53]
 [127  31]]
Acc:  0.392  Prec:  0.369  Recall:  0.196  F1: 0.256
SVM
Confusion Matrix : 
[[ 50  88]
 [ 25 133]]
Acc:  0.618  Prec:  0.602  Recall:  0.842  F1: 0.702
Decision Tree
Confusion Matrix : 
[[ 62  76]
 [ 35 123]]
Acc:  0.625  Prec:  0.618  Recall:  0.778  F1: 0.689
Random Forest
Confusion Matrix : 
[[ 78  60]
 [ 40 118]]
Acc:  0.662  Prec:  0.663  Recall:  0.747  F1: 0.702
NN
Confusion Matrix : 
[[138   0]
 [158   0]]
Acc:  0.466  Prec:  0.0  Recall:  0.0  F1: 0.0
k = ----------  3
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 87  51]
 [127  31]]
Acc:  0.3

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random Forest
Confusion Matrix : 
[[ 83  55]
 [ 43 115]]
Acc:  0.669  Prec:  0.676  Recall:  0.728  F1: 0.701
NN
Confusion Matrix : 
[[ 81  57]
 [ 38 120]]
Acc:  0.679  Prec:  0.678  Recall:  0.759  F1: 0.716
k = ----------  4
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 92  46]
 [129  29]]
Acc:  0.409  Prec:  0.387  Recall:  0.184  F1: 0.249
SVM
Confusion Matrix : 
[[ 54  84]
 [ 24 134]]
Acc:  0.635  Prec:  0.615  Recall:  0.848  F1: 0.713
Decision Tree
Confusion Matrix : 
[[ 64  74]
 [ 35 123]]
Acc:  0.632  Prec:  0.624  Recall:  0.778  F1: 0.693
Random Forest
Confusion Matrix : 
[[ 83  55]
 [ 43 115]]
Acc:  0.669  Prec:  0.676  Recall:  0.728  F1: 0.701
NN
Confusion Matrix : 
[[ 82  56]
 [ 40 118]]
Acc:  0.676  Prec:  0.678  Recall:  0.747  F1: 0.711
k = ----------  5
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[ 93  45]
 [127  31]]
Acc

...

Random Forest
Confusion Matrix : 
[[ 96  42]
 [ 47 111]]
Acc:  0.699  Prec:  0.725  Recall:  0.703  F1: 0.714
NN
Confusion Matrix : 
[[ 91  47]
 [ 30 128]]
Acc:  0.74  Prec:  0.731  Recall:  0.81  F1: 0.769
k = ----------  56
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[64 74]
 [94 64]]
Acc:  0.432  Prec:  0.464  Recall:  0.405  F1: 0.432
SVM
Confusion Matrix : 
[[ 81  57]
 [ 31 127]]
Acc:  0.703  Prec:  0.69  Recall:  0.804  F1: 0.743
Decision Tree
Confusion Matrix : 
[[ 88  50]
 [ 38 120]]
Acc:  0.703  Prec:  0.706  Recall:  0.759  F1: 0.732
Random Forest
Confusion Matrix : 
[[ 92  46]
 [ 47 111]]
Acc:  0.686  Prec:  0.707  Recall:  0.703  F1: 0.705
NN
Confusion Matrix : 
[[  0 138]
 [  0 158]]
Acc:  0.534  Prec:  0.534  Recall:  1.0  F1: 0.696
k = ----------  57
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[66 72]
 [94 64]]
Acc:  0.439  P

NN
Confusion Matrix : 
[[ 82  56]
 [ 33 125]]
Acc:  0.699  Prec:  0.691  Recall:  0.791  F1: 0.737
k = ----------  69
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[74 64]
 [93 65]]
Acc:  0.47  Prec:  0.504  Recall:  0.411  F1: 0.453
SVM
Confusion Matrix : 
[[ 82  56]
 [ 30 128]]
Acc:  0.709  Prec:  0.696  Recall:  0.81  F1: 0.749
Decision Tree
Confusion Matrix : 
[[ 87  51]
 [ 38 120]]
Acc:  0.699  Prec:  0.702  Recall:  0.759  F1: 0.729
Random Forest
Confusion Matrix : 
[[ 92  46]
 [ 32 126]]
Acc:  0.736  Prec:  0.733  Recall:  0.797  F1: 0.764
NN
Confusion Matrix : 
[[ 90  48]
 [ 37 121]]
Acc:  0.713  Prec:  0.716  Recall:  0.766  F1: 0.74
k = ----------  70
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[72 66]
 [95 63]]
Acc:  0.456  Prec:  0.488  Recall:  0.399  F1: 0.439
SVM
Confusion Matrix : 
[[ 81  57]
 [ 29 129]]
Acc:  0.709  Prec:  0.

/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SVM
Confusion Matrix : 
[[ 81  57]
 [ 31 127]]
Acc:  0.703  Prec:  0.69  Recall:  0.804  F1: 0.743
Decision Tree
Confusion Matrix : 
[[ 88  50]
 [ 36 122]]
Acc:  0.709  Prec:  0.709  Recall:  0.772  F1: 0.739
Random Forest
Confusion Matrix : 
[[ 93  45]
 [ 37 121]]
Acc:  0.723  Prec:  0.729  Recall:  0.766  F1: 0.747
NN
Confusion Matrix : 
[[ 85  53]
 [ 35 123]]
Acc:  0.703  Prec:  0.699  Recall:  0.778  F1: 0.737
k = ----------  75
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[69 69]
 [95 63]]
Acc:  0.446  Prec:  0.477  Recall:  0.399  F1: 0.434
SVM
Confusion Matrix : 
[[ 81  57]
 [ 30 128]]
Acc:  0.706  Prec:  0.692  Recall:  0.81  F1: 0.746
Decision Tree
Confusion Matrix : 
[[ 84  54]
 [ 37 121]]
Acc:  0.693  Prec:  0.691  Recall:  0.766  F1: 0.727
Random Forest
Confusion Matrix : 
[[ 89  49]
 [ 36 122]]
Acc:  0.713  Prec:  0.713  Recall:  0.772  F1: 0.742
NN
Confusion Matrix : 
[[ 87  51]
 [ 36 122]]
Acc:  0.7

Random Forest
Confusion Matrix : 
[[ 92  46]
 [ 30 128]]
Acc:  0.743  Prec:  0.736  Recall:  0.81  F1: 0.771
NN
Confusion Matrix : 
[[ 92  46]
 [ 40 118]]
Acc:  0.709  Prec:  0.72  Recall:  0.747  F1: 0.733
k = ----------  88
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[68 70]
 [94 64]]
Acc:  0.446  Prec:  0.478  Recall:  0.405  F1: 0.438
SVM
Confusion Matrix : 
[[ 82  56]
 [ 28 130]]
Acc:  0.716  Prec:  0.699  Recall:  0.823  F1: 0.756
Decision Tree
Confusion Matrix : 
[[ 90  48]
 [ 41 117]]
Acc:  0.699  Prec:  0.709  Recall:  0.741  F1: 0.724
Random Forest
Confusion Matrix : 
[[ 91  47]
 [ 36 122]]
Acc:  0.72  Prec:  0.722  Recall:  0.772  F1: 0.746
NN
Confusion Matrix : 
[[  0 138]
 [  0 158]]
Acc:  0.534  Prec:  0.534  Recall:  1.0  F1: 0.696
k = ----------  89
Classes found :  ['helpful' 'not_helpful']

Classes converted to integers : [1 1 0 ... 0 1 1]
NB
Confusion Matrix : 
[[68 70]
 [94 64]]
Acc:  0.446  P